<a href="https://colab.research.google.com/github/Hirofrench/Computer_Science_Assignment/blob/main/Computer_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment preperation

In [2]:
# Code below is to make drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install needed packages
!pip install nltk
!pip install gensim
!pip install python-Levenshtein
!pip install searchgrid
!pip install bayesian-optimization
!pip install hyperopt


In [4]:
import numpy as np
import pandas as pd
import json
import random
import re
import nltk
import time
import warnings
import Levenshtein as lev
import numpy as np
import matplotlib.pyplot as plt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.cluster import AgglomerativeClustering
from nltk import ngrams
from collections import Counter
from statistics import mean
from math import *
from itertools import combinations
from collections import defaultdict
from sklearn.utils import shuffle
from random import choices, seed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
d = open('/content/drive/MyDrive/Com_assignment/TVs-all-merged.json')
df = json.load(d)
start = time.perf_counter()


bestbuy = 0
newegg = 0
amazon = 0
thenerds = 0
count = 0

common_brands = np.array([])
shops = np.array([])
df_clean = df

title_remove_words = np.array(["diag.", "[", "]", "open box:", "refurbished", "best buy", ")", "(", ","])

for value in df_clean:
  for tv in df_clean[value]:
    #get vector with al shop names
    if tv['shop'] not in shops:
      shops = np.append(shops, tv['shop'])
    
    #clean title values
    tv['title'] = tv['title'].lower()
    tv['title'] = tv['title'].replace(" hz", "hz")
    tv['title'] = tv['title'].replace("hertz", "hz")
    tv['title'] = tv['title'].replace("-hz", "hz")
    tv['title'] = tv['title'].replace(" hertz", "hz")

    tv['title'] = tv['title'].replace("inches", "inch")
    tv['title'] = tv['title'].replace("-inch", "inch")
    tv['title'] = tv['title'].replace(" inch", "inch")
    tv['title'] = tv['title'].replace("\"", "inch")
    tv['title'] = tv['title'].replace(" inches", "inch")

    for word in title_remove_words:
      tv['title'] = tv['title'].replace(word, "")
    for word in shops:
      tv['title'] = tv['title'].replace(word, "")

    #as sometimes typos are made, i.e. we find 3112inch instead of 31-1/2 inch
    for word in tv['title'].split():
      if 'inch' in word and len(re.sub("[^\d\.]", "", word))>3:
        tv['title'] = tv['title'].replace(word, "")
      if any(letter == "." for letter in word):
        pos_dot = word.find(".")
        #check if not a decimal number
        if pos_dot != 0 and not word[pos_dot-1].isdigit():
          tv['title'] = tv['title'].replace(word, "")
    

    #count tv's per shop
    if 'bestbuy.com' in tv.values():
      bestbuy+=1
    elif 'newegg.com' in tv.values():
      newegg+=1
    elif 'amazon.com' in tv.values():
      amazon+=1
    elif 'thenerds.net' in tv.values():
      thenerds+=1
    count +=1
    for feature in tv['featuresMap'].keys():
      #clean feature valuies
      tv['featuresMap'][feature] = tv['featuresMap'][feature].lower()
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(" hz", "hz")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("-hz", "hz")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("hertz", "hz")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(" hertz", "hz")

      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("inches", "inch")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("-inch", "inch")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(" inch", "inch")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("\"", "inch")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(" inches", "inch")
      
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("lbs", "lb")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("lbs.", "lb")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("lb.", "lb")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("pounds", "lb")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("pound", "lb")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(" lb", "lb")

      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(")", "")
      tv['featuresMap'][feature] = tv['featuresMap'][feature].replace("(", "")

      #make decimal representation universal i.e. 7-1/2inch should be 7.5inch
      feature_value = ""
      for word in tv['featuresMap'][feature].split():
        value = word
        if "http://" in word:
          tv['featuresMap'][feature] = tv['featuresMap'][feature].replace(word, "")
        if any(letter=="-" for letter in word):
          pos_bar = word.find("-")
          if any(letter=="/" for letter in word):
            pos_slash = word.find("/")
            if len(word) > pos_bar and pos_slash>pos_bar:
              
              if word[pos_bar-1].isdigit() and word[pos_bar+1].isdigit():
                string = ""
                for i in range(len(word)):
                  if word[i].isalpha() and not word[i].isdigit():
                    string = string + word[i]
                dec_number = re.sub("[^\d\.]", "", word[pos_slash+1:])
                dec_string = str(word[:pos_slash+1]+dec_number)
                num, denom = dec_string[pos_bar+1:].split('/')

                try:
                  fraction = str(round(float(int(word[:pos_bar]) + float(int(num)/int(denom))),1))
                except:
                  num, denom = dec_string[pos_bar:].split('/')
                  fraction = str(round(float(int(word[:pos_bar-1]) + float(int(num)/int(denom))),1))
                value = str(fraction+string)
              else:
                ;
            else:
              ;
          else:
            ;
        else:
          
          #Convert all decimal numbers to 1 decimal numbers to have a more universal set of measures
          if any(letter=="." for letter in word):
            pos_dot = word.find(".")
            if pos_dot>0 and pos_dot<len(word)-2:
              if word[pos_dot-1].isdigit() and word[pos_dot+1].isdigit() and word[pos_dot+2].isdigit():
                string = ""
                for i in range(len(word)):
                  if word[i].isalpha() and not word[i].isdigit():
                    string = string + word[i]
                try:
                  dec_number = re.sub("[^\d\.]", "", word[pos_dot+1:])
                  new_number = str(round(float(str(word[:pos_dot-1]+"."+dec_number)),1))
                  value = str(new_number+string)
                except:
                  ; 
        if not feature_value:
          feature_value = value
        else:
          feature_value = str(feature_value + " " + value)
      tv['featuresMap'][feature] = feature_value
      #print(feature_value)

    #make uniform description for brand
    if ('Brand' in tv['featuresMap'].keys() and tv['featuresMap']['Brand'] not in common_brands):
      common_brands = np.append(common_brands, tv['featuresMap']['Brand'])
    elif 'Brand Name' in tv['featuresMap'].keys():
      tv['featuresMap']['Brand'] = tv['featuresMap'].pop('Brand Name')
      if tv['featuresMap']['Brand'] not in common_brands:
        common_brands = np.append(common_brands, tv['featuresMap']['Brand'])
    elif 'Brand Name:' in tv['featuresMap'].keys():
      tv['featuresMap']['Brand'] = tv['featuresMap'].pop('Brand Name:')
      if tv['featuresMap']['Brand'] not in common_brands:
        common_brands = np.append(common_brands, tv['featuresMap']['Brand'])
common_brands = np.append(common_brands, ["dynex", "insignia"])
#create universal screensize if possible
search_key = 'Screen Size'
for value in df_clean:
  for tv in df_clean[value]:
    res = dict(filter(lambda item: search_key in item[0], tv['featuresMap'].items()))

    if res:
      screen_size = list(res.values())[0]
      screen_update = screen_size[0:6]
      screen_size_update = re.sub("[^\d\.]", "", screen_update)
      screen_size_update = str(screen_size_update+"inch")
      tv['featuresMap'].update({"Universal Screensize" : screen_size_update})
    
    elif 'inch' in tv['title']:
      for word in tv['title'].split():
        if 'inch' in word:
          tv['featuresMap'].update({"Universal Screensize" : word}) 
    else:
      tv['featuresMap'].update({"Universal Screensize" : "not found"})


#some brand names are displayed in different names, check this
wrong_brand = np.array([])
good_brand = np.array([])
for j in common_brands:
  for k in common_brands:
    if j in k and len(j)<len(k):
      wrong_brand = np.append(wrong_brand, k)
      good_brand = np.append(good_brand, j)

#get missing brands from titlenames
index = 0
for value in df_clean:
  for tv in df_clean[value]:  

    tv.update({"Index": index})
    index +=1
    for j in common_brands:
      if ('Brand' not in tv['featuresMap'].keys()):
        if j in tv['title']:
          tv['featuresMap'].update({"Brand": j})
          break
        elif common_brands[-1] == j:
          tv['featuresMap'].update({"Brand": "uncommon"})
    counter = -1
    for k in wrong_brand:
      counter+=1
      if tv['featuresMap']['Brand'] == k:
        tv['featuresMap']['Brand'] = good_brand[counter]
        break
end = time.perf_counter()
print("Number of unique Tv's: " + str(len(df)))
print("Number of TV's sold by BestBuy: " + str(bestbuy))
print("Number of TV's sold by NewEgg: " + str(newegg))
print("Number of TV's sold by Amazon: " + str(amazon))
print("Number of TV's sold by TheNerds: " + str(thenerds))
print("Data size: " + str(count))
print("Number of duplicates: " + str(count-len(df)))
print(f"Data transformation took {end - start:0.4f} seconds")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:145: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Number of unique Tv's: 1262
Number of TV's sold by BestBuy: 773
Number of TV's sold by NewEgg: 668
Number of TV's sold by Amazon: 163
Number of TV's sold by TheNerds: 20
Data size: 1624
Number of duplicates: 362
Data transformation took 0.7656 seconds


### Product representation: Model words

In [7]:
#This function extracts the modelwords from the data and creates the signaturematrix
def modelwords_func(df_clean):
  warnings.filterwarnings("ignore")
  global model_words
  global model_words_title
  global model_words_features
  start = time.perf_counter()
  model_words_title = np.array([])
  model_words_features = np.array([])
  must_contain_mw = ['hz', 'lbs', 'inch']
  for value in df_clean:
    for tv in df_clean[value]:
      mw = tv['title'].split()
      for word in mw:
        if any(letter.isalpha() for letter in word):
          if any(not letter.isalpha() for letter in word):
            if word not in model_words_title:
              model_words_title = np.append(model_words_title, word)
        elif any(letter.isdigit() for letter in word):
          if any(not letter.isdigit() for letter in word):
            if word not in model_words_title:
              model_words_title = np.append(model_words_title, word)
      '''
      #own representation
      for features in tv['featuresMap'].values():
        mw_features = features.split()
        for i in must_contain_mw:
          for word in mw_features:
            if (i in word and word not in model_words_features):
              if any(letter.isdigit() for letter in word):
                model_words_features = np.append(model_words_features, word)
      '''
      for features in tv['featuresMap'].values():
        mw_features = features.split()
        for word in mw_features:
          if any(letter == "." for letter in word):
            if any(letter.isdigit() for letter in word):
              if re.sub("[^\d\.]", "", word) not in model_words_features:
                model_words_features = np.append(model_words_features, re.sub("[^\d\.]", "", word))

  

  print("   Amount of modelwords from title: " + str(len(model_words_title)))
  print("   Amount of modelwords from featuresmap: " + str(len(model_words_features)))
  model_words = np.concatenate((model_words_title, model_words_features))
  #develop the signature matrix
  df_update = df_clean
  for value in df_update:
    for tv in df_update[value]:
      prod_rep_title = np.zeros(len(model_words_title), dtype=int)
      prod_rep_features = np.zeros(len(model_words_features), dtype=int)
      for j, mw in enumerate(model_words_title):
        #if (mw in tv['title']):
        if (mw in tv['title'] or mw in tv['featuresMap'].values()):
          prod_rep_title[j] = 1
        
      
      for k, mw in enumerate(model_words_features):
        for j in tv['featuresMap'].values():
          if mw in j:
            prod_rep_features[k] = 1

          
      prod_representation = np.concatenate((prod_rep_title, prod_rep_features))
      tv.update({"Product Representation" : prod_representation})
  np.savetxt('model_words_test.csv', model_words, delimiter=',', fmt ='% s')
  #np.savetxt('test.csv', binary_vector, delimiter=',', fmt ='% s')
  end = time.perf_counter()
  print(f"   Extracting modelwords and constructing product representation took {end - start:0.4f} seconds")
  print("-"*40)
  return df_update, model_words


### Min-Hashing and Locality Sensitivity Hashing

In [19]:
#This function performs the minhashing
def min_hash_func(df_clean, model_words, unique_keys):
  #min-Hashing
  start = time.perf_counter()
  hashes = 200
  signature_matrix = np.zeros((hashes,len(unique_keys)))
  index = np.array([])

  #create array of indices of model words
  for i in range(len(model_words)):
    index = np.append(index, i).astype(int)

  #start minhashing  
  for i in range(hashes):
    #make permutation every hash
    perm = shuffle(index, random_state = i)
    product=0
    for value in df_clean:
      for tv in df_clean[value]:
        counter=-1
        for mw in range(len(model_words)):
          counter+=1
          permutated_index = perm[counter]
          if tv['Product Representation'][permutated_index] == 1:
            signature_matrix[i][product] = counter
            product+=1
            counter=-1
            #if value of 1 is found, next product should be checked, therefore break
            break
  signature_matrix = signature_matrix.astype(int)
  np.savetxt('sig_test.csv', signature_matrix, delimiter=',', fmt ='% s')
  end = time.perf_counter()
  print(f"   Minhashing took {end - start:0.4f} seconds")
  print("-"*40)     
  return signature_matrix

#This function performs the LSH
def lsh_func(signature_matrix, considered_items):
  #locality Sensitive Hashing (LSH)
  start = time.perf_counter()
  n, d = signature_matrix.shape
  b = 60
  r = n/b

  #check if condition holds
  assert n==b*r, "    Something goes wrong within LSH"
  hashbuckets = defaultdict(set)
  bands = np.array_split(signature_matrix, b, axis=0)
  
  for i, band in enumerate(bands):
    for j in range(d):
      band_id = tuple(list(band[:,j])+ [str(i)])
      #print(band_id)
      hashbuckets[band_id].add(j)
  candidate_pairs = set()

  for bucket in hashbuckets.values():
    if len(bucket) >= 2:
      possible_pairs = combinations(bucket,2)
      for pair in possible_pairs:
        candidate_pairs.add(pair)

  #remove duplicates
  dissim_matrix = np.full((d,d), -1*np.inf)
  counter = 0
  cand_pairs_no_duplicate = candidate_pairs
  remove_set = set()
  for i in cand_pairs_no_duplicate:
    first = i[0]
    second = i[1]
    if dissim_matrix[first][second] == 1:
      counter+=1
      remove_set.add(i)
    dissim_matrix[first][second] = 1
    dissim_matrix[second][first] = 1

  #remove duplicate sets
  for i in remove_set:
    cand_pairs_no_duplicate.remove(i)
  end = time.perf_counter()
  print("   " + str(counter) + " pairs have been removed as these were double in de candidate pairs")
  print(f"   Locality Sensitivity Hashing took {end - start:0.4f} seconds")
  print("-"*40)
  return cand_pairs_no_duplicate

### Clustering Algorithm

In [9]:
#This function cleans the candidate pairs obtained from LSH based on certain product details
def clean_candidate_pairs_func(candidate_pairs, df_update):
  #remove candidate pairs from same shop and from different brands
  start = time.perf_counter()
  print("   Initial candidate pairs: " + str(len(candidate_pairs)))
  clean_cand_pairs = candidate_pairs
  remove_set = set()
  count_brand_removal = 0
  count_screensize_removal = 0
  count_keyvalue_removal = 0
  for i in clean_cand_pairs:
    screensize = np.array([])
    brand = np.array([])
    pair_index = np.array([i[0],i[1]])
    pair_dict = dict()
    tv_1 = dict()
    tv_2 = dict()
    key = 'first'
    for j in pair_index:
      for value in df_update:
        for tv in df_update[value]:
          if value == j:
            pair_dict.update({key: tv}) 

            #find all key value pairs of which the value is either yes or no, or the value represents a digit of length 1
            for key, feature in pair_dict['first']['featuresMap'].items():
              if 'yes' in feature:
                tv_1.update({key: 'yes'})
              elif 'no' in feature:
                tv_1.update({key: 'no'})
              elif len(feature)==1 and feature.isdigit():
                tv_1.update({key:feature })
            if (len(pair_dict.keys())>1):
              for key, feature in pair_dict['second']['featuresMap'].items():
                if 'yes' in feature:
                  tv_2.update({key: 'yes'})
                elif 'no' in feature:
                  tv_2.update({key: 'no'})
                elif len(feature)==1 and feature.isdigit():
                  tv_2.update({key:feature })
            
            screensize = np.append(screensize, re.sub("inch", "", tv['featuresMap']['Universal Screensize']))
            brand = np.append(brand, tv['featuresMap']['Brand'])
            key = 'second'
            break
        else:
          continue
        break

    #case 1: both brands are uncommon, therefor could be the same brand
    if brand[0] == "uncommon" and brand[1] == "uncommon":
      ;
    #case 2: brands are not similar, therefor remove
    elif brand[0] != brand[1]:
      remove_set.add(i)
      count_brand_removal+=1
      # assert pair_dict['first']['featuresMap']['Brand'] != pair_dict['second']['featuresMap']['Brand'],  \
      # "   Error: Two tv's of same brand have been removed."
      # assert pair_dict['first']['modelID'] != pair_dict['second']['modelID'], \
      # "   Error: Two same tv's have been removed, due to brand." 

    #remove pairs which do not have the same screensize. due to rounding, we accept maximum 0.5 difference
    elif screensize[0] == "not found" or screensize[1] == "not found":
      ;
    elif abs(float(screensize[0]) - float(screensize[1])) >0.5:
      remove_set.add(i)
      count_screensize_removal+=1
      # assert pair_dict['first']['modelID'] != pair_dict['second']['modelID'], \
      # "   Error: Two same tv's have been removed, due to screensize."
    elif bool(tv_1)==True and bool(tv_2)==True:
      for key1, feature1 in tv_1.items(): 
        for key2, feature2 in tv_2.items(): 
          if key1==key2 and feature1!=feature2:
            remove_set.add(i)
            count_keyvalue_removal+=1

  #remove all candidate pairs of which the shop is the same, under the assumption that one shop does not sell the same tv twice
  for i in remove_set:
    clean_cand_pairs.remove(i)
  
  end = time.perf_counter() 

  print("   Removal different brand: " + str(count_brand_removal))
  print("   Removal due to different screensize: " + str(count_screensize_removal))
  print("   Removal due to different key-value pairs: " + str(count_keyvalue_removal))
  print("   New number of candidate pairs: " + str(len(clean_cand_pairs)))
  print(f"   Pair cleaning took {end - start:0.4f} seconds")
  print("-"*40)
  return clean_cand_pairs




In [10]:
def tdif ():
  dissim_matrix_update = dissim_matrix
  for i in range(len(dissim_matrix)):
    for j in range(len(dissim_matrix)):
      if dissim_matrix[i][j] == 1:
        titles = [df_update[i][0]['title'],df_update[j][0]['title']]
        vect = TfidfVectorizer(min_df=1, stop_words="english")
        tfidf = vect.fit_transform(titles) 
        pairwise_similarity = tfidf * tfidf.T 
        pairwise_similarity.toarray() 
        dissim_matrix_update[i][j] = pairwise_similarity.toarray()[0][1]
  correct = 0
  not_correct = 0
  for i in range(len(dissim_matrix_update)):
    for j in range(len(dissim_matrix_update)):
      if dissim_matrix[i][j]>0.2 and i<j:
        if df_update[i][0]['modelID'] == df_update[j][0]['modelID']:
          correct+=1
        else:
          not_correct+=1
  print(correct)
  print(not_correct)

In [11]:
#This function calculates the dice coefficient
def dice_coef(set1, set2):
  count = 0
  for i in set1:
    if i in set2:
      count+=1    
  dice =  2*count/(len(set1)+len(set2))
  return dice

#This function creates all the n-grams and eventually finds the final pairs based on the similarities between the n-grams
def ngram_func(cand_pairs, df_update, shops, common_brands, alpha, beta, theta, gamma,mu,unique_keys):
  cleaned_pairs = cand_pairs 
  dissim_matrix = np.full((len(df_update),len(df_update)), -1*np.inf)
  dissim_kv = dissim_matrix
  
  #these words should be removed from the title as they are meaningless or already filtered on
  rem_words = np.append(common_brands, ["inch", "best buy", "Diag.", "amazon"]) 
  remove_words = np.append(rem_words, shops)
  for i in cleaned_pairs:
    first_tv = df_update[i[0]][0]
    second_tv = df_update[i[1]][0] 
    tv_value1 = dict()
    tv_value2 = dict()
    tv_key1 = dict()
    tv_key2 = dict()
    first = i[0]
    second = i[1]
    
    #n grams for key value pairs
    for key, feature in first_tv['featuresMap'].items():
      n_grams_kv_1 = []
      ngram_key1 = []
      first_tv['featuresMap'][key] = first_tv['featuresMap'][key].replace(",", "")
      first_tv['featuresMap'][key] = first_tv['featuresMap'][key].replace("/", "")
      first_tv['featuresMap'][key] = first_tv['featuresMap'][key].replace("\ ", "")
      for word in feature.split():
        for j in range(len(word)-2): 
          if len(word)>=3:   
              n_grams_kv_1.append(word[j: j + 3])

      #make ngrams from keys
      for word in key.split():
        for j in range(len(word)-2):
          if len(word)>=3:
            ngram_key1.append(word[j: j + 3])

      #exclude information about brand, screensize etc. as these are already filtered on and will shift similarity towards each other
      if key != 'Brand' and 'inch' not in feature and len(n_grams_kv_1)>0 and feature != 'yes' and feature != 'no':
        tv_value1.update({key: n_grams_kv_1})
        tv_key1.update({key:ngram_key1})
     
    for key, feature in second_tv['featuresMap'].items():
      n_grams_kv_2 = []
      ngram_key2 = []
      second_tv['featuresMap'][key] = second_tv['featuresMap'][key].replace(",", "")
      second_tv['featuresMap'][key] = second_tv['featuresMap'][key].replace("/", "")
      second_tv['featuresMap'][key] = second_tv['featuresMap'][key].replace("\ ", "")
      for word in feature.split():
        for j in range(len(word)-2): 
          if len(word)>=3:
            n_grams_kv_2.append(word[j: j + 3])

      #make ngrams from keys
      for word in key.split():
        for j in range(len(word)-2):
          if len(word)>=3:
            ngram_key2.append(word[j: j + 3])

      if key != 'Brand' and 'inch' not in feature and len(n_grams_kv_2)>0 and feature != 'yes' and feature != 'no':
        tv_value2.update({key: n_grams_kv_2})
        tv_key2.update({key:ngram_key2})

    dice_kv = []
    match = dict()
    match_dice = dict()
    for key, ngram in tv_key1.items():
      for key2, ngram2 in tv_key2.items():
        dice_key = dice_coef(ngram, ngram2)
        #check for similarity in keys
        if dice_key >= beta:
          #create dictionary with matching keys which have a dice coefficient of minimum beta
          match.update({key:key2})
          match_dice.update({key:dice_key})
    weight = 0
    #find dice coefficient for values of 'similar' keys
    for key, ngram in tv_value1.items():    
      for match1, match2 in match.items():      
        if key==match1:
          ngram2 = tv_value2[match2]
          dice_it = dice_coef (ngram, ngram2)
          dice_key = match_dice[match1]
          weight = weight+dice_key
          dice_kv.append(dice_it*dice_key)
    if len(dice_kv)>0:
      dissim_kv[first][second] = sum(dice_kv)/weight
      dissim_kv[second][first] = sum(dice_kv)/weight
      #print(dissim_kv[first][second])
    #if no matching keys assign 0
    else:
      dissim_kv[first][second] = 0
      dissim_kv[second][first] = 0

    
  
  dissim_matrix_update = np.full((len(df_update),len(df_update)), -1*np.inf)
  #n grams for title
  dissim_tmwm = np.full((len(df_update),len(df_update)), -1*np.inf)
  for i in cleaned_pairs:
    first = i[0]
    second = i[1]
    first_tv = df_update[i[0]][0]
    second_tv = df_update[i[1]][0] 
    title_1 = first_tv['title']
    title_2 = second_tv['title']
    for word in remove_words:
      for title_word in title_1.split():
        title_1 = title_1.replace(",", "")
        title_1 = title_1.replace("/", "")
        title_1 = title_1.replace("\ ", "")
        title_1 = title_1.replace(":", "")
        if word in title_word or len(title_word)==1: 
          title_1 = title_1.replace(title_word, "")
      for title_word in title_1.split():
        if word in title_word or len(title_word)==1: 
          title_2 = title_2.replace(title_word, "")
    n_grams_1 = []
    n_grams_2 = []
    for word in title_1.split():
      for i in range(len(word)-2): 
        n_grams_1.append(word[i: i + 3])
    for word in title_2.split():
      for i in range(len(word)-2): 
        n_grams_2.append(word[i: i + 3])
    
    dice_sim = dice_coef(n_grams_1, n_grams_2)
    dissim_matrix_update[first][second] = dice_sim
    dissim_matrix_update[second][first] = dice_sim

    #calculate tmwm similarity
    tmwm_sim = tmwm_func(title_1, title_2, df_update, gamma, first_tv, second_tv)
    dissim_tmwm[first][second] = tmwm_sim
    dissim_tmwm[second][first] = tmwm_sim


  #make a dissimilarity matrix based on key value pair and title
  dissim_mix = np.full((len(df_update),len(df_update)), -1*np.inf)
  for i in range(0, len(dissim_mix)-1):
    for j in range(i+1, len(dissim_mix)):
      if dissim_kv[i][j]==0:
        dissim_mix[i][j] = dissim_matrix_update[i][j]
        dissim_mix[j][i] = dissim_matrix_update[i][j]
      if dissim_kv[i][j]>=0:
        average_value = alpha*dissim_matrix_update[i][j] + mu*dissim_kv[i][j] + (1-alpha-mu)*dissim_tmwm[i][j]
        #average_value = alpha*dissim_kv[i][j] + (1-alpha)*dissim_tmwm[i][j]
        dissim_mix[i][j] = average_value
        dissim_mix[j][i] = average_value

  correct =0
  not_correct=0

  #return final set of candidate pairs
  remove_set = set()
  
  for k in cleaned_pairs:
    i = k[0]
    j = k[1]
    if dissim_mix[i][j]>=theta:
      if df_update[i][0]['modelID'] == df_update[j][0]['modelID']:
        correct+=1
      elif dissim_mix[i][j]>=0:
        not_correct+=1
    else:
      remove_set.add(k)
  clean_pairs = set()
  for i in cleaned_pairs:
    clean_pairs.add(i)
  for i in remove_set:
    clean_pairs.remove(i)
  return clean_pairs, dissim_kv, dissim_matrix_update


In [12]:
#def tmwm_func(cand_pairs):



def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = sqrt(sum1) * sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)

def tmwm_func(first_title, second_title, df_update, gamma, first_tv, second_tv):
  title_1 = text_to_vector(first_title)
  title_2 = text_to_vector(second_title)
  cos_sim = get_cosine(title_1, title_2)
  if cos_sim < gamma:
    total_sim = 0
  else:
    total_sim = cos_sim
  mw_2 = np.array([])
  mw_1 = np.array([]) 
  for word in model_words_title:
    if word in first_tv['title']:
      mw_1 = np.append(mw_1, word)
    if word in second_tv['title']:
      mw_2 = np.append(mw_2, word)
  for j in mw_1:
    for k in mw_2:
      if any(letter.isdigit() for letter in j) and any(letter.isdigit() for letter in k) and \
      any(letter.isalpha() for letter in j) and any(letter.isalpha() for letter in k):
        non_numeric_1 = re.sub(" \d+", " ", j) 
        non_numeric_2 = re.sub(" \d+", " ", k)
        bigram_1 = []
        bigram_2 = []
        for t in range(len(non_numeric_1)-1):
          if len(non_numeric_1)>=2:
            bigram_1.append(non_numeric_1[t: t + 2])
        for t in range(len(non_numeric_2)-1):
          if len(non_numeric_2)>=2:
            bigram_2.append(non_numeric_2[t: t + 2])
        if bigram_1 and bigram_2:
          dice = dice_coef(bigram_1, bigram_2)
        else:
          dice = 0
        if dice>0.9:
          numeric_1 = re.findall('[0-9]+', j)
          numeric_2 = re.findall('[0-9]+', k)
          if numeric_1 != numeric_2:
            total_sim = 0
            break
          
  return total_sim


In [13]:
def split_pairs(candidate_pairs,nr_groups):
  nr_pgroup = floor(len(candidate_pairs)/nr_groups)
  rest = len(candidate_pairs) - nr_pgroup*nr_groups
  new_groups = dict()
  counter = 1
  pairs = set()
  for j, pair in enumerate(candidate_pairs):
    pairs.add(pair)
    if j== counter*nr_pgroup:
      new_groups.update({counter:pairs})
      counter+=1
      pairs= set()
    elif j==len(candidate_pairs):
      new_groups.update({counter:pairs})
  return new_groups

In [14]:
#Set the parameter grids of the bayesian optimization
#set startvalue, so no errors are given
alpha = 0
bayes_parameters = {'alpha' : hp.uniform('alpha',0,1),
                    'beta' : hp.uniform('beta',0,1),
                    'theta' : hp.uniform('theta',0,1),
                    'gamma' : hp.uniform('gamma',0,1),
                    'mu' : hp.uniform('mu',0,1-alpha)}

In [15]:
#this function should be minimized by the bayesian optimization, is minimizes the loss function, which is -1*f1-value
def bayesgrid_search(space):
  alpha = space['alpha']
  beta = space['beta']
  theta = space['theta']
  gamma = space['gamma']
  mu = space['mu']
  cand_pairs = pairs_bayes
  total_pairs = 0
  for t in cand_pairs:
    first = t[0]
    second = t[1]
    if bayes_df[first][0]['modelID'] == bayes_df[second][0]['modelID']:
      total_pairs+=1
  pairs_update = ngram_func(cand_pairs, bayes_df, shops, common_brands, alpha, beta, theta, gamma,mu,unique_keys)
  f1_it, p_quality, p_complete = f1_func(pairs_update[0], bayes_df, graph_total_pairs, unique_keys)

  return {'loss': -f1_it, 'pq': p_quality, 'pc': p_complete, 'status': STATUS_OK}

In [21]:
#This function runs the whole algorithm and returns either all the parameters if it's the train set, otherwise the f1 value if it's the test set

def algorithm_func(df_alg, unique_keys, shops, common_brands, train_test,alpha,beta,theta,gamma,mu,i):
  it_start = time.perf_counter()
  global graph_total_pairs
  #check possible pairs for this bootstrap    
  print("   Amount of tv's in bootstrap " + str(i+1) + ": " + str(len(unique_keys)))
  modelID_pairs, total_pairs = duplicate_count(unique_keys, df_alg)
  graph_total_pairs = total_pairs
  #make variables global, such that they can be found during bayesian optimization
  global pairs_bayes
  global bayes_df

  #get modelwords from current sample
  bayes_df, model_words = modelwords_func(df_alg)
  #get signature matrix with minhashing
  signature_matrix = min_hash_func(bayes_df, model_words, unique_keys)
  #get candidate pairs with lsh
  candidate_pairs = lsh_func(signature_matrix, bayes_df)
  fraction_comparisons = len(candidate_pairs)/nCr(len(unique_keys),2)
  #get f1 measure pure on LSH
  f1_lsh, pq_lsh, pc_lsh = f1_func(candidate_pairs, bayes_df, total_pairs, unique_keys)
  #clean candidate pairs
  pairs_bayes = clean_candidate_pairs_func(candidate_pairs, bayes_df)

  #clustering with grid search
  if train_test == 'train':
    trials = Trials()
    best_params = fmin(fn=bayesgrid_search,
            space=bayes_parameters,
            algo=tpe.suggest,
            trials=trials,
            return_argmin=False, # Return categorical vars as strings instead of indices
            max_evals=20,
            rstate= np.random.RandomState(i))
    best_alpha = best_params['alpha']
    best_beta = best_params['beta']
    best_theta = best_params['theta']
    best_gamma = best_params['gamma']
    best_mu = best_params['mu'] 
    best_f1  = [x['result']['loss'] for x in trials.trials]
    index_min = best_f1.index(min(best_f1))
    best_pq  = [x['result']['pq'] for x in trials.trials][index_min]
    best_pc  = [x['result']['pc'] for x in trials.trials][index_min]
  elif train_test == 'test':
    final_pairs,dissim_kv, dissim_matrix_update = ngram_func(pairs_bayes, bayes_df, shops, common_brands, alpha, beta, theta, gamma,mu, unique_keys )
    f1_iteration, p_quality, p_complete = f1_func(final_pairs, bayes_df, total_pairs, unique_keys)
  
  it_end = time.perf_counter()
  print(f"   Iteration took {it_end - it_start:0.4f} seconds")
  print("*"*60)
  if train_test == 'train':
    return best_alpha, best_beta, best_theta, best_gamma, best_mu, best_f1, f1_lsh, pq_lsh, pc_lsh
    #return best_alpha, best_beta, best_theta, best_gamma, best_mu,  best_f1, f1_lsh, pq_lsh, pc_lsh
  else:
    return f1_iteration, p_quality, p_complete

### Evaluation


In [16]:
#This is a simple function which calculates nCr
def nCr(n,r):
  f = factorial
  return f(n) / (f(r) * f(n-r))

#This function counts per dataset howmuch possible pairs there are in the set
def duplicate_count(unique_keys, df_bootstrap):
  unique_items = unique_keys.astype(int)
  modelID_pairs = dict()
  model_test = np.array([])
  
  for value in df_bootstrap:
    for tv in df_bootstrap[value]:
      model_test = np.append(model_test, tv['modelID'])
  model_unique = np.unique(model_test)

  total_pairs = 0
  for i in model_unique:
    counter = 0
    for value in df_bootstrap:
      for tv in df_bootstrap[value]:
        if i == tv['modelID']:
          counter+=1

    if counter>=2:
      possible_pairs = nCr(counter,2)
    else:
      possible_pairs = 0
    total_pairs = total_pairs + possible_pairs
    assert counter>0, "   Duplicate count mistake"
    modelID_pairs.update({i: possible_pairs})

  print("   Amount of pairs: " + str(int(total_pairs)))
  print("-"*40)
  return modelID_pairs, total_pairs

def f1_func(cleaned_pairs, df_f1, total_pairs, unique_keys):
  correct = 0
  not_correct = 0
  f1_measure = 0
  pair_quality = 0
  pair_completeness = 0
  for i in cleaned_pairs:
    pair_index = np.array([i[0],i[1]])
    pair_model = np.array([])
    
    for j in pair_index:
      for value in df_f1:
        for tv in df_f1[value]:
          if value == j:
            pair_model = np.append(pair_model, tv['modelID'])
            break
        else:
          continue
        break
    if pair_model[0] == pair_model[1]:
      correct+=1
      assert df_f1[i[0]][0]['modelID'] == df_f1[i[1]][0]['modelID'], \
      "   Mistake in classifying correct pairs"
    else:
      not_correct+=1
      assert df_f1[i[0]][0]['modelID'] != df_f1[i[1]][0]['modelID'], \
      "   Mistake in classifying correct pairs"
  if correct>0 and not_correct>0:
    pair_quality = correct/(correct+not_correct)
    pair_completeness = correct / total_pairs
    fraction_comparisons = len(cleaned_pairs) / nCr(len(unique_keys),2)
    f1_measure = (2*pair_quality*pair_completeness)/(pair_quality + pair_completeness)
    f1_alt = correct/(correct + 0.5*(not_correct+(total_pairs-correct)))
    print("   Correct pair: " + str(correct))
    print("   Not correct pair: " + str(not_correct))
    # print("   Pair quality: " + str(pair_quality))
    # print("   Pair completeness: " + str(pair_completeness))
    # print("   Fraction of comparisons: " + str(fraction_comparisons))
    # print("   F1 value: " + str(f1_measure))
    # print("   F1 alternative value: " + str(f1_alt))
  else:
    print("   No comparisons are made.")
  print("-"*40)
  return f1_measure, pair_quality, pair_completeness



### Bootstrapping

In [ ]:
nr_bootstraps = 20
index_dict = np.array([])
f1_measure = np.array([])
pc_measure = np.array([])
pq_measure = np.array([])
alpha_overal = dict()
beta_overal = dict()
theta_overal = dict()
mu_overal = dict()
gamma_overal = dict()
f1_overal = dict()
f1_lsh = dict()
pq_lsh = dict()
pc_lsh = dict()
#create array of indices of model words
for i in range(count):
  index_dict = np.append(index_dict, i).astype(int)

for i in range(nr_bootstraps):
  total_start = time.perf_counter()
  print ("Iteration: " + str(i+1))
  df_train = dict()
  df_test = dict()
  np.random.seed(i)
  global unique_keys
  keys = np.random.choice(a=index_dict,replace = True, size=len(index_dict)*1)
  unique_keys = np.unique(keys)
  dissim_matrix = np.zeros((len(unique_keys), len(unique_keys)))
  test_key = np.array([])
  #find values for test set
  for value in df_clean:
    for tv in df_clean[value]: 
      if tv['Index'] not in unique_keys:
        test_key = np.append(test_key, int(tv['Index']))
  test_key = test_key.astype(int)

  #make dictionary with test set
  for index, j in enumerate(test_key):
    for value in df_clean:
      for tv in df_clean[value]:
        if tv['Index'] == j:
          df_test.update({index:[tv]})
          break
      else:
        continue
      break

  for index, j in enumerate(unique_keys):
    for value in df_clean:
      for tv in df_clean[value]:
        if tv['Index'] == j:
          df_train.update({index:[tv]})
          break
      else:
        continue
      break
  

  #obtain best parameters in train set
  alpha, beta, theta, gamma, mu,  f1_train, f1_hash, pq_hash, pc_hash = algorithm_func(df_train, unique_keys, shops, common_brands, 'train', 0,0,0,0,0,i)

  #save all values obtainend in dictionary
  f1_max = -1*min(f1_train)
  alpha_overal.update({i:alpha})
  beta_overal.update({i:beta})
  theta_overal.update({i:theta})
  mu_overal.update({i:mu})
  gamma_overal.update({i:gamma})
  f1_overal.update({i:f1_max})
  f1_lsh.update({i:f1_hash})
  pc_lsh.update({i:pc_hash})
  pq_lsh.update({i:pq_hash})
  print("   TRAIN OUTPUT")
  print(alpha_overal)
  print(beta_overal)
  print(theta_overal)
  print(gamma_overal)
  print(mu_overal)
  print(f1_overal)
  print("*"*60)
  
  unique_keys = test_key
  #get results for test set
  f1_test, pq_test, pc_test = algorithm_func(df_test, unique_keys, shops, common_brands, 'test', alpha, beta, theta,gamma,mu,i)

  print("   TEST OUTPUT")
  # #evaluate the models 
  f1_measure = np.append(f1_measure, f1_test)
  pc_measure = np.append(pc_measure, pc_test)
  pq_measure = np.append(pq_measure, pq_test)
  print(f1_measure)
  print(pc_measure)
  print(pq_measure)

  total_end = time.perf_counter()
  print(f"   Iteration took {total_end - total_start:0.4f} seconds")
  print("*"*60)
print("Average F1 value: " +  str(mean(f1_measure)))
print("Average Pair Quality: " + str(mean(pq_measure)))
print("Average Pair Completeness: " + str(mean(pc_measure)))